In [1]:
!pip install -q datasets


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


# Import Libraries

In [2]:
from datasets import load_dataset
import re

# Load dataset

In [3]:
dataset = load_dataset('timdettmers/openassistant-guanaco')

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [4]:
# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

In [5]:
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})


In [7]:
for i in range(3):
    print(dataset[i])

{'text': '### Human: Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo?### Assistant: Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país.'}
{'text': '### Human: Самый великий человек из всех живших на планете?### Assistant: Для начала нужно выбрать критерии величия

In [15]:
# Define a function to transform the data
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split("###")
    # print("segments: ", segments)

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()
        # print("Human text: ",human_text)

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()
            # print("assistant_text: ", assistant_text)

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST]</s>')
        
            
    return {'text': ''.join(reformatted_segments)}

In [16]:
# Apply the transformation
transformed_dataset = dataset.map(transform_conversation)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
transformed_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [19]:
for i in range(5):
    print(transformed_dataset[i])

{'text': '<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'}
{'text': '<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии величия человека

# Push to hugging face

In [ ]:
# transformed_dataset.push_to_hub("guanaco-llama2-1k")